In [ ]:
# 1️⃣ IMPORT REQUIRED LIBRARIES
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import joblib
import os


In [ ]:

# 2️⃣ LOAD PREPROCESSED DATASET
# Adjust the path to your dataset location
df = pd.read_csv("D:/FYP/data/datasets/final_dataset_02.csv")


In [ ]:

# 3️⃣ DEFINE FEATURES (X) AND LABEL (y)
X = df.drop(columns=['label'])         # All input features
y = df['label'].astype(int)            # Binary target variable (0 = Normal, 1 = Attack)


In [ ]:

# 4️⃣ TRAIN-TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:

#  5️⃣ INITIALIZE CATBOOST CLASSIFIER
catboost_model = CatBoostClassifier(
    iterations=500,                  # Number of boosting rounds
    learning_rate=0.1,               # Step size
    depth=6,                         # Tree depth
    loss_function='Logloss',        # Binary classification loss
    eval_metric='AUC',              # Evaluation metric
    early_stopping_rounds=50,       # Stop training if no improvement
    verbose=100,                    # Print progress
    random_seed=42
)


In [ ]:

#  6️⃣ TRAIN THE MODEL
catboost_model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    use_best_model=True,
    verbose=100
)


In [ ]:

#  7️⃣ MAKE PREDICTIONS
y_pred = catboost_model.predict(X_test)
y_pred_proba = catboost_model.predict_proba(X_test)[:, 1]  # Probability for ROC-AUC


In [ ]:

#  8️⃣ EVALUATE PERFORMANCE
print("\n✅ Model Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred_proba))
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:

# 9️⃣ SAVE TRAINED MODEL IN .PKL FORMAT
os.makedirs("trained_models", exist_ok=True)  # Create 'trained_models' folder if not exists
joblib.dump(catboost_model, "models/trained_models/catboost_intrusion_detection.pkl")
print("✅ CatBoost model saved successfully at: models/trained_models/catboost_intrusion_detection.pkl")
